# Multi Speaker Speech to Text

Alle notwendige Pakete importieren (siehe requirements.txt)

In [3]:
from pyannote.audio import Pipeline
from pydub import AudioSegment
import whisper
import os

Definieren welches Audio-Recording genutzt werden soll.

In [ ]:
audio_file = "ldn-topic.wav"

## Initialize pipeline

speaker-diarization pipeline initialisieren. Dafuer wird eine Registierung bei dem Model und ein Access Token fuer Huggingface benoetigt:

1. visit hf.co/pyannote/speaker-diarization and hf.co/pyannote/segmentation and accept user conditions (only if requested)
2. visit hf.co/settings/tokens to create an access token (only if you had to go through 1.)

Huggingface Token muss in Umgebungsvariable `HF_TOKEN` gesetzt werden

In [4]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=os.environ["HF_TOKEN"])

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


## Audio "parsen"

Audio file parsen. `diarization` beinhaltet die abschnitte und den Sprecher

In [5]:
diarization = pipeline(audio_file, num_speakers=2)

## Whisper Model laden

In [7]:
whispermodel = whisper.load_model("base")

## Audio splitten und chunks zu text umwandeln

In [12]:
audio = AudioSegment.from_wav(audio_file)
i = 1

f = open("transscript.txt", "a")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    newAudio = audio[turn.start*1000:turn.end*1000]
    newAudio.export(f"part{i}.wav", format="wav")
    result = whispermodel.transcribe(f"part{i}.wav")
    f.write(f'{speaker}:{result["text"]}\n')
    print(f"start={turn.start} stop={turn.end} speaker_{speaker}")
    i = i + 1

f.close()

start=0.4978125 stop=21.0009375 speaker_SPEAKER_01
start=21.0009375 stop=44.06906250000001 speaker_SPEAKER_00
start=44.06906250000001 stop=59.948437500000004 speaker_SPEAKER_01
